In [16]:
# -*- coding:utf-8 -*-  

from sys import path
import time
import numpy as np
import tensorflow as tf
import extract_mnist
import scipy.io as sio

# Parameter
batch_size = 64
isTrain = True

#初始化单个卷积核上的参数
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

#初始化单个卷积核上的偏置值
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#输入特征x，用卷积核W进行卷积运算，strides为卷积核移动步长，
#padding表示是否需要补齐边缘像素使输出图像大小不变
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

#对x进行最大池化操作，ksize进行池化的范围，
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='VALID')

# average pooling
def avg_pool_2x2(x):
    return tf.nn.avg_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='VALID')

def input_poisson(x_image, level):
    #print x_image
    x_image = (x_image+0.5) * level
    
    w = x_image.shape[0]
    h = x_image.shape[1]
    img = np.random.poisson(x_image, (w, h))
    #img = x_image
    img = img / level
    #print img
    return img

    
#定义会话
sess = tf.InteractiveSession()

#声明输入图片数据，类别
x = tf.placeholder('float',[None,784])
y_ = tf.placeholder('float',[None,10])
#输入图片数据转化
x_image = tf.reshape(x,[-1,28,28,1])

#第一层卷积层，初始化卷积核参数、偏置值，该卷积层5*5大小，一个通道，共有20个不同卷积核
#[filter_height, filter_width, in_channels, out_channels]
W_conv1 = weight_variable([5, 5, 1, 20])
#进行卷积操作，并添加relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)) #withoutbias
#进行最大池化
h_pool1 = avg_pool_2x2(h_conv1)

#同理第二层卷积层
W_conv2 = weight_variable([5,5,20,50])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2))
h_pool2 = avg_pool_2x2(h_conv2)

#全连接层
#权值参数
W_fc1 = weight_variable([4*4*50,500])
#将卷积的产出展开
h_pool2_flat = tf.reshape(h_pool2,[-1,4*4*50])
#神经网络计算，并添加relu激活函数
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1))

#Dropout层，可控制是否有一定几率的神经元失效，防止过拟合，训练时使用，测试时不使用
#Dropout计算

#输出层，使用softmax进行多分类
W_fc2 = weight_variable([500,10])
h_fc2 = tf.matmul(h_fc1, W_fc2)
y_conv=tf.maximum(tf.nn.softmax(h_fc2),1e-30)

#代价函数
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
#使用Adam优化算法来调整参数
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#测试正确率
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#保存参数
saver = tf.train.Saver()

#所有变量进行初始化
sess.run(tf.global_variables_initializer())

#获取mnist数据
mnist_data_set = extract_mnist.MnistDataSet('../mnist/')
te_images,test_labels = mnist_data_set.test_data()
test_images = input_poisson(te_images, 255.0)
#test_images = te_images + 0.5

#进行训练
if isTrain:
    start_time = time.time()
    for i in xrange(10000):
        #获取训练数据
        xs, batch_ys = mnist_data_set.next_train_batch(batch_size)
        batch_xs = input_poisson(xs, 50.0)
        #print batch_xs.shape[0]
        #每迭代100个 batch，对当前训练数据进行测试，输出当前预测准确率
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch_xs, y_: batch_ys})
            print "step %d, training accuracy %g"%(i, train_accuracy)
            #计算间隔时间
            end_time = time.time()
            print 'time: ',(end_time - start_time)
            start_time = end_time
        #训练数据
        train_step.run(feed_dict={x: batch_xs, y_: batch_ys})


    #保存参数
    if not tf.gfile.Exists('output/model'):
        tf.gfile.MakeDirs('output/model')
    save_path = saver.save(sess, "output/model/model.ckpt")
    print "Model saved in file: ", save_path

    #保存网络权值
    if not tf.gfile.Exists('output/weights'):
        tf.gfile.MakeDirs('output/weights')
    conv1_weights = sess.run(W_conv1)
    conv2_weights = sess.run(W_conv2)
    ip1_weights = sess.run(W_fc1)
    ip2_weights = sess.run(W_fc2)
    sio.savemat('output/weights/lenet_avg_pooling.mat', {'conv1_weights':conv1_weights, 
                                                        'conv2_weights':conv2_weights, 
                                                        'ip1_weights':ip1_weights, 
                                                        'ip2_weights':ip2_weights})
else:
    saver.restore(sess, "output/model/model.ckpt")
# 输出整体测试数据的情况
avg = 0
for i in xrange(200):
    avg += accuracy.eval(feed_dict={x: test_images[i*50:i*50+50], y_: test_labels[i*50:i*50+50]})
avg/=200
print "test accuracy %g"%avg

conv1_out = sess.run(h_conv1, feed_dict={x: test_images[0,:].reshape((1,784))})

print conv1_weights[:,:,0,0]

tmp = conv1_out[0,:,:,0] * 255
print tmp.astype(int)

# 关闭会话
#sess.close()


step 0, training accuracy 0.078125
time:  0.0263640880585
step 100, training accuracy 0.890625
time:  0.508879899979
step 200, training accuracy 0.90625
time:  0.381189107895
step 300, training accuracy 0.921875
time:  0.388643026352
step 400, training accuracy 0.921875
time:  0.375592947006
step 500, training accuracy 0.875
time:  0.384376049042
step 600, training accuracy 0.9375
time:  0.384478807449
step 700, training accuracy 0.859375
time:  0.382387161255
step 800, training accuracy 0.890625
time:  0.442514896393
step 900, training accuracy 0.9375
time:  0.427339076996
step 1000, training accuracy 0.9375
time:  0.430077075958
step 1100, training accuracy 0.953125
time:  0.464365959167
step 1200, training accuracy 0.96875
time:  0.418670892715
step 1300, training accuracy 0.953125
time:  0.448665142059
step 1400, training accuracy 0.921875
time:  0.455335855484
step 1500, training accuracy 0.96875
time:  0.387542963028
step 1600, training accuracy 0.96875
time:  0.413632154465
step

In [31]:
tmp = conv1_out[0,:,:,0] * 255
print tmp.astype(int)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0  10  26  37  65  48  30  24   0   1   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0  37  61  85 127  94 100  96  83  97  75  66  74  76  80  78  55
   34   0   0   0   0   0]
 [  0   0  19  36  60  93  63  57  29  40  79  82  77  86  73  91  97  66
   50  18   0   0   0   0]
 [  0   0  10  42  93  65   0   0   0  20   0   0   0  25  13  52  83  38
   17   0   0   0   0   0]
 [  0   0  16  37  76  12  12   0   0   0   0   0   0   0   0  40  51  31
    7   0   0   0   0   0]
 [  0   0   0  41  37  33   0   0   0   0   0   0   0   0   8  38  47  78
    0   0   0   0   0   0]
 [  0   0   0   8  10   4   4   5   9   9   0   0   0   9  28  76 109 138
    0   0   0   0

In [17]:
tmp = conv1_out[0,:,:,0] * 255
print tmp.astype(int)


[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0  11  40  66  81  62  52  22  17   3   2   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0  46 114 182 201 216 211 168 146 114 116 101 109 103 105 103  81
   49  19  11   4   0   0]
 [  0   0  62 143 194 218 255 282 281 286 279 277 271 271 266 260 254 217
  151  82  46  18   0   0]
 [  0   0  46  55  38  35  77 106 156 198 244 255 280 270 266 281 302 280
  177 116  75  34   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   7   0  10  66 134 129
   49  55  52  21   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  35   3   0
    0   0   0   0

In [15]:
fc2_out = sess.run(h_fc1, feed_dict={x: test_images[0,:].reshape((1,784))})
tmp = fc2_out * 255
print tmp.astype(int)

[[   0    0   58    0    0    0   72  229  301   54  269    0  321   50
   396  124   33   25    0   24   29  491  363  349    0   64    0  543
     0    0    0    0  532    0   80    0    0    0    0    0    0    0
     0  675  487    0  182  433    0  106    0  368  238  990    0    0
    71  549  126  270    0  386  219    0    0    0   96    0    0  305
   636    0   58  666    0  215  493  146  805  426  408  173    0   49
   344    0  377  284  103  622    0 1067    0  484  249  954  255  392
     0    0  713    0    0    0    0  315    0    0  351    0    0    2
     0  437    0    0   77    0    0  500    0   98  419    0    0  209
   348    0    0    0  320    0  408  424    0  230   63  157    0    0
     0   23    0   32  345    0  111  967  473  175  803  367    0  403
   385  253  121   66    0  249  205  718  736    0  370  874    0  264
   542   32  123  354   33    0  365  535  461  749    0  623  969    0
   239    0  733    0    0    0  944    0  428  333  382  318   